# ERCOT Market Analysis (PTP Obligation Bid Offers)
*StrategyWise for Southern Power Company, 2019.Q4*

*Data Analysis Performed by Robin Sanders*

In [8]:
# Import necessary packages
import pandas as pd
from pandas import DataFrame, read_csv
import numpy as np
import calendar
import glob
from datetime import datetime
from dateutil.parser import parse
import os
import urllib.request
import seaborn as sns

# Matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#Regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
import math
%matplotlib inline

#see all columns/rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# Part 1: Prepare the Data------------------------------------------------------------

## Import and Clean Data
###  strip/clean column headers, datetime conversion on:

##### Environmental: Market Data, Locational Marginal Pricing, Nodes
*Third Party Environmental: NOAA Hourly Weather Data*
##### Transactional: PTP Obligation Bid Offers
*Third Party Transactional: Daily NASDAQ, DOWJONES, ETF prices*

In [2]:
#import ERCOT
market_df = pd.read_excel('OneDrive_1_10-22-2019/ercot_market_data.xlsx', sheet_name = 'ercot_market_data')
lmp_df = pd.read_csv('OneDrive_1_10-22-2019/ercotlmp.csv')
nodes_df = pd.read_excel('OneDrive_1_10-22-2019/ercot_nodes.xlsx')
ptp_bids_df = pd.concat([pd.read_csv(f) for f in glob.glob('OneDrive_1_10-22-2019/PTPObligationBids/*.csv')], ignore_index = True)

#strip/clean column headers
ptp_bids_df.columns = ptp_bids_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace(' - ',' ')
ptp_bids_df = ptp_bids_df.rename(columns={'ptp_bid_-_mw':'ptp_bid_mw','ptp_bid_-_price':'ptp_bid_price'})
market_df.columns = market_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
lmp_df.columns = lmp_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
nodes_df.columns = nodes_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')

#drop duplicate/unnecessary/sparse columns
market_df = market_df.drop(columns=['datetime','year'])
lmp_df = lmp_df.drop(columns=['datetime','year'])
nodes_df = nodes_df.drop(columns=['iso','weatherstationid','first_dart_date','last_dart_date','equipment','voltage',
                                    'substation','nodetype','zoneid','objectid'])
#convert marketday feature from datetime type to string type for merging
market_df['marketday'] = market_df['marketday'].dt.strftime('%m/%d/%Y')

#import 3rd party data
weather_df_1 = pd.read_csv('additional_data/weather_data_1.csv',low_memory=False)
weather_df_2 = pd.read_csv('additional_data/weather_data_2.csv',low_memory=False)
weather_df_3 = pd.read_csv('additional_data/weather_data_3.csv',low_memory=False)
nasdaq_df = pd.read_csv('additional_data/nasdaq_data.csv')
etf_df = pd.read_csv('additional_data/etf_data.csv')
dowjones_df = pd.read_csv('additional_data/dow_jones_data.csv')

#strip/clean column headers
weather_df_1.columns = weather_df_1.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
weather_df_2.columns = weather_df_2.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
weather_df_3.columns = weather_df_3.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
nasdaq_df.columns = nasdaq_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
etf_df.columns = etf_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
dowjones_df.columns = dowjones_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')

#Select HOURLY weather data 
weather_df_1_trim = weather_df_1[['station','date','hourlydrybulbtemperature','hourlyrelativehumidity',
                                'hourlystationpressure','hourlywinddirection','hourlywindspeed']]
weather_df_2_trim = weather_df_2[['station','date','hourlydrybulbtemperature','hourlyrelativehumidity',
                                'hourlystationpressure','hourlywinddirection','hourlywindspeed']]
weather_df_3_trim = weather_df_3[['station','date','hourlydrybulbtemperature','hourlyrelativehumidity',
                                'hourlystationpressure','hourlywinddirection','hourlywindspeed']]

#rename columns, drop 'close' column and use adjusted close column 'adj_close'
nasdaq_df = nasdaq_df.rename(columns={'date':'nasdaq_date','open':'nasdaq_open','high':'nasdaq_high',
                                     'low':'nasdaq_low','close':'nasdaq_close','adj_close':'nasdaq_adj_close','volume':'nasdaq_volume'})
nasdaq_df = nasdaq_df.drop(columns='nasdaq_close')
etf_df = etf_df.rename(columns={'date':'etf_date','open':'etf_open','high':'etf_high','low':'etf_low',
                                     'close':'etf_close','adj_close':'etf_adj_close','volume':'etf_volume'})
etf_df = etf_df.drop(columns='etf_close')
dowjones_df = dowjones_df.rename(columns={'date':'dowjones_date','open':'dowjones_open','high':'dowjones_high',
                                     'low':'dowjones_low','close':'dowjones_close','adj_close':'dowjones_adj_close','volume':'dowjones_volume'})
dowjones_df = dowjones_df.drop(columns='dowjones_close')

### Displaying Content of ERCOT PTP Offers / 3rd Party datasets

#### market_df

In [6]:
print(market_df.shape)
market_df.head()

(6575, 14)


,gr_panhandle_wind_stwpf,panhandle_ercot_rt_generic_constraints,ercot_total_resource_cap_out,ercot_gen_resource,ercot_rtload,ercot_original_load_forecast,ercot_wind_rti,ercot_wind_stwpf_orig,ercot_total_resource_cap_out.1,ercot_renew_resource_cap_out,hourending,marketday,peaktype,month
0,3713.4,3836,8525,50955.2,37081.44343,37031.792114,14317.82,13102.1,8525,1545,1,01/01/2019,OFFPEAK,JANUARY
1,3692.8,3836,8533,51139.2,37258.98993,37486.635803,14126.46,12025.1,8533,1588,2,01/01/2019,OFFPEAK,JANUARY
2,3587.2,3829,8533,51649.4,37300.18780,37394.205750,13686.93,11819.6,8533,1588,3,01/01/2019,OFFPEAK,JANUARY
3,3494.1,3826,8533,51116.7,37423.54347,37487.759094,13345.24,11499.3,8533,1569,4,01/01/2019,OFFPEAK,JANUARY
4,3426.3,3831,8533,50534.0,37895.21228,38759.001221,13238.17,11146.3,8533,1569,5,01/01/2019,OFFPEAK,JANUARY


#### nodes_df

In [7]:
print(nodes_df.shape)
nodes_df.head()

(771, 3)


,nodename,zone,nearest_weatherstation
0,ACACIA_UNIT1,WEST,TX - Marfa/Municipal
1,AEEC,WEST,TX - Lubbock/Intl
2,AMISTAD_ALL,SOUTH,TX - San Angelo/Mathis
3,AMOCOOIL_CC1,HOUSTON,TX - Houston/Intercontinental
4,AMOCOOIL_CC2,HOUSTON,TX - Houston/Intercontinental


#### lmp_df

In [8]:
print(lmp_df.shape)
lmp_df.head()

(3823733, 7)


,rtlmp,dalmp,hourending,marketday,peaktype,month,settlementpoint
0,13.0175,20.58,1,01/01/2019,OFFPEAK,JANUARY,AEEC
1,13.4000,14.48,1,01/01/2019,OFFPEAK,JANUARY,AMISTAD_ALL
2,14.2350,20.07,1,01/01/2019,OFFPEAK,JANUARY,AMOCOOIL_CC1
3,14.2350,20.07,1,01/01/2019,OFFPEAK,JANUARY,AMOCOOIL_CC2
4,14.2350,20.07,1,01/01/2019,OFFPEAK,JANUARY,AMOCO_PUN1


#### ptp_bids_df

In [14]:
print(ptp_bids_df.shape)
ptp_bids_df.head()

(21026292, 9)


,delivery_date,hour_ending,qse_name,settlement_point_source,settlement_point_sink,ptp_bid_-_mw,ptp_bid_price,bid_id,multi-hour_block_indicator
0,08/02/2019,1,QALTU2,FERMI_ALL,AMISTAD_ALL,10.0,0.0,47079354,N
1,08/02/2019,1,QALTU2,FERMI_ALL,AMISTAD_ALL,10.0,0.4,47079355,N
2,08/02/2019,1,QALTU2,FERMI_ALL,AMISTAD_ALL,10.0,0.7,47079356,N
3,08/02/2019,1,QALTU2,FERMI_ALL,AMISTAD_ALL,10.0,1.0,47079357,N
4,08/02/2019,1,QALTU2,FERMI_ALL,AMISTAD_ALL,10.0,1.4,47079358,N


#### Example Weather: weather_df_1_trim

In [10]:
print(weather_df_1_trim.shape)
weather_df_1_trim.head()

(129963, 7)


,station,date,hourlydrybulbtemperature,hourlyrelativehumidity,hourlystationpressure,hourlywinddirection,hourlywindspeed
0,72267023042,2019-01-01T00:00:00,25,81.0,26.69,030,22
1,72267023042,2019-01-01T00:53:00,24,84.0,26.74,040,21
2,72267023042,2019-01-01T01:53:00,23,81.0,26.77,020,16
3,72267023042,2019-01-01T02:53:00,22,82.0,26.77,040,21
4,72267023042,2019-01-01T03:53:00,22,82.0,26.78,050,21


#### Example Stock: nasdaq_df

In [11]:
print(nasdaq_df.shape)
nasdaq_df.head()

(133, 6)


,nasdaq_date,nasdaq_open,nasdaq_high,nasdaq_low,nasdaq_adj_close,nasdaq_volume
0,2019-01-02,6506.910156,6693.709961,6506.879883,6665.939941,2261800000
1,2019-01-03,6584.770020,6600.209961,6457.129883,6463.500000,2607290000
2,2019-01-04,6567.140137,6760.689941,6554.240234,6738.859863,2579550000
3,2019-01-07,6757.529785,6855.600098,6741.399902,6823.470215,2507550000
4,2019-01-08,6893.439941,6909.580078,6795.859863,6897.000000,2380290000


## Trim Bid Offer Data:  Top 10 Leaders in Timeframe 1.2.19 - 7.12.19

In [3]:
#Top 10 leaders
lead_ptp_offers = ptp_bids_df.loc[ptp_bids_df['qse_name'].isin(['QLUMN','QNRGTX','QDCENG','QREUEL','QSHELL',
                                                                    'QDIRE','QPREC','QMONT','QWOLFP','QTIOS'])]
#Select Data within timeline presented in Jeff's Power BI Dashboard (Jan 2, 2019 - July 12, 2019)
lead_ptp_offers['date'] = pd.to_datetime(lead_ptp_offers['delivery_date'])
mask = (lead_ptp_offers['date'] >= '2019-01-02') & (lead_ptp_offers['date'] < '2019-07-13')
lead_ptp_offers = lead_ptp_offers.loc[mask]

#lead_ptp_offers is the base of the merging section to create a model-ready dataset

/Users/robinsanders/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


## Join Market Data, LMP, Nodes & Leader PTP Offers Tables = data_ercot

In [6]:
#Join Market & Location Pricing & Nodes Data
lmp_nodes = lmp_df.merge(nodes_df, how='left', left_on='settlementpoint', right_on='nodename') #merge with nodes
lmp_nodes = lmp_nodes.drop(columns='nodename') #drop unnecessary columns
#Join PTP Offers with joined Market/Price df
lead_ptp_offers = lead_ptp_offers.rename(columns={'delivery_date':'marketday','hour_ending':'hourending',
                                                  'settlement_point_source':'settlementpoint_src','settlement_point_sink':'settlementpoint_snk'})
awards_Src = lead_ptp_offers.merge(lmp_nodes.add_prefix('src_'), how = 'left', 
                                          left_on = ['marketday','hourending','settlementpoint_src'],
                                          right_on = ['src_marketday','src_hourending','src_settlementpoint'])
data_ercot = awards_Src.merge(lmp_nodes.add_prefix('snk_'), how = 'left', 
                                          left_on = ['marketday','hourending','settlementpoint_snk'],
                                          right_on = ['snk_marketday','snk_hourending','snk_settlementpoint'])
data_ercot.loc[(data_ercot.snk_nearest_weatherstation == 'TM - Nuevo Laredo/Intl'),'snk_nearest_weatherstation']='TX - Laredo/Intl'
data_ercot.loc[(data_ercot.src_nearest_weatherstation == 'TM - Nuevo Laredo/Intl'),'src_nearest_weatherstation']='TX - Laredo/Intl'
#Nuevo Laredo Intl is close enough to Laredo Intl to use the same weatherstation
data_ercot = data_ercot.merge(market_df, how = 'left', on = ['marketday','hourending'])
data_ercot = data_ercot.fillna(0.0)
data_ercot.loc[(data_ercot.snk_zone == 0.0),'snk_zone']='Other'

#data_ercot is a merged table and consists of ERCOT DATA: PTP Offers, market, LMP, and nodes data

In [9]:
#Confirm no loss of primary PTP transaction data
print(lead_ptp_offers.shape)
print(data_ercot.shape)

(4856190, 10)
(4856190, 40)


## Join 3rd Party Data to data_ercot =  data

In [10]:
#Merge Stock Data together
stocks_df = nasdaq_df.merge(etf_df, how='left',left_on='nasdaq_date',right_on='etf_date')
stocks_df = stocks_df.merge(dowjones_df, how='left',left_on='nasdaq_date',right_on='dowjones_date')
stocks_df = stocks_df.drop(columns=['dowjones_date','etf_date'])
stocks_df = stocks_df.rename(columns={'nasdaq_date':'date'})
stocks_df['date'] = pd.to_datetime(stocks_df['date'],infer_datetime_format=True)
stocks_df['date'] = stocks_df['date'].dt.strftime('%m/%d/%Y')

In [11]:
#Merge Weather together
weather_df = pd.concat([weather_df_1_trim, weather_df_2_trim,weather_df_3_trim])
weather_df['hourlydrybulbtemperature'] = pd.to_numeric(weather_df['hourlydrybulbtemperature'], errors='coerce', downcast=None)
weather_df['hourlystationpressure'] = pd.to_numeric(weather_df['hourlystationpressure'], errors='coerce', downcast=None)
weather_df['hourlywinddirection'] = pd.to_numeric(weather_df['hourlywinddirection'], errors='coerce', downcast=None)
weather_df['hourlywindspeed'] = pd.to_numeric(weather_df['hourlywindspeed'], errors='coerce', downcast=None)

weather_df['date'] = pd.to_datetime(weather_df['date'],infer_datetime_format=True)
weather_df['hourending'] = [d.time() for d in weather_df['date']]
mask = (weather_df['date'] >= '2019-01-02') & (weather_df['date'] < '2019-07-13')
weather_df = weather_df.loc[mask]
hours = [math.ceil((t.hour * 60 + t.minute) / 60) for t in weather_df['hourending']]
weather_df['hour'] = hours
weather_df['hour']= weather_df['hour'].apply(str).apply(int)
weather_df = weather_df.loc[(weather_df['hour'] > 0)]
weather_df['date'] = weather_df['date'].dt.strftime('%m/%d/%Y')
c_maxes = weather_df.groupby(['station', 'date','hour']).hourending.transform(max)
weather_df = weather_df.loc[weather_df.hourending == c_maxes]
weather_df['station'] = weather_df['station'].map({72267023042: 'TX - Lubbock/Intl',
                                                                 72251012924: 'TX - Corpus Christi/Intl',
                                                                 72266013962: 'TX - Abilene/Municipal', 
                                                                 72250012919: 'TX - Brownsville/Intl', 
                                                                 72351013966: 'TX - Wichita Falls/Sheppard AFB',
                                                                 72261022010: 'TX - Del Rio/Intl',
                                                                 72265023023: 'TX - Midland-Odessa',
                                                                 72253012921: 'TX - San Antonio/Intl',
                                                                 72363023047: 'TX - Amarillo/Intl',
                                                                 72248013957: 'LA - Shreveport/Regional',
                                                                 72263023034: 'TX - San Angelo/Mathis',
                                                                 72265623040: 'TX - Wink/Winkler County',
                                                                 72258013960: 'TX - Dallas/Love Field',
                                                                 72243012960: 'TX - Houston/Intercontinental',
                                                                 72261823091: 'TX - Fort Stockton',
                                                                 72252012907: 'TX - Laredo/Intl',
                                                                 74641013975: 'OK - Gage/Shattuck',
                                                                 72259303985: 'TX - Dallas-Fort Worth/Intl'})

In [12]:
#Merge ERCOT data with weather and stock data 
weather_df=weather_df.drop_duplicates(keep='first')
data = data_ercot.merge(weather_df.drop_duplicates(['date','hour','station']).add_prefix('src_'), 
                       how = 'left', 
                       left_on = ['marketday','hourending','src_nearest_weatherstation'], 
                       right_on=['src_date','src_hour','src_station'])
data = data.merge(weather_df.drop_duplicates(['date','hour','station']).add_prefix('snk_'), 
                       how = 'left', 
                       left_on = ['marketday','hourending','snk_nearest_weatherstation'], 
                       right_on=['snk_date','snk_hour','snk_station'])
data = data.merge(stocks_df, how = 'left', left_on='marketday', right_on='date')
data = data.fillna(0.00)
data.loc[(data.src_zone == 0.0),'src_zone']='Other'
data =data.drop(columns=['bid_id','date_x','src_hourending_x','src_settlementpoint','snk_hourending_x','snk_month','snk_settlementpoint','src_station','src_hourending_y','snk_station','snk_hourending_y','date_y','src_marketday','snk_marketday','src_date','snk_date','src_hour','snk_hour', 'src_nearest_weatherstation','snk_nearest_weatherstation','ercot_total_resource_cap_out.1'])

#data is now the final merged table, consists of all ERCOT and 3rd party data
#Ready to model.

In [13]:
#Confirm no loss of primary PTP transaction data
print(data_ercot.shape)
print(data.shape)


(4856190, 40)
(4856190, 53)


In [ ]:
data.to_csv("ptp_obligation_bidoffers_market_lmp_node_thirdparty_model_ready.csv")